**Cache Augmented Generation (CAG)**

In [ ]:
!pip install transformers torch mysql-connector-python streamlit flask pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.7 MB/s eta 0:00:00


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import mysql.connector
import torch


In [ ]:
model_name = 'google/flan-t5-base' # st3ml llama2-8b-instruct 7sn
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
'''
we use the snipet code below to connect with MySQL DB
and fetch all data related with the evenments
'''
# Connect to MySQL DB
# def fetch_reservations(user, password, host, db_name):
#   db_config = {
#       'user': user,
#       'password': password,
#       'host': host,
#       'database': db_name,
#       'raise_on_warnings': True
#   }

#   conn = mysql.connector.connect(**db_config)
#   cursor = conn.cursor()


#   cursor.execute("select * from évènements")
#   reservations = cursor.fetchall()

#   conn.close()

#   documents = []
#   for reservation in reservations:
#     id, date, description, _, lieu, nom, utilisateur_id = reservation
#     doc = f"Reservation ID is : {id}, in the date {date}, the reservation description is {description}, and this happen in {lieu}, Created by {utilisateur_id}"
#     documents.append(doc)

#     return documents
'''
in our case we are using a ready-data to test out CAG implementation,
we used CAG because this data is static annualy and less computation
'''
def fetch_reservations():
    records = [
        (1, '2025-01-15', 'Annual Tech Conference focused on AI and Cybersecurity', 'https://example.com/tech2025.jpg', 'San Francisco', 'Tech Innovators 2025', 201),
        (2, '2025-02-10', 'A music festival featuring top artists from around the globe', 'https://example.com/musicfest.jpg', 'London', 'Global Music Fest', 202),
        (3, '2025-03-22', 'An exhibition showcasing modern art from emerging artists', 'https://example.com/artexpo.jpg', 'Paris', 'Modern Art Expo', 203),
        (4, '2025-04-05', 'Marathon for raising awareness on climate change', 'https://example.com/marathon.jpg', 'New York', 'Green Earth Marathon', 204),
        (5, '2025-05-18', 'International book fair with author signings and workshops', 'https://example.com/bookfair.jpg', 'Berlin', 'World Book Fair 2025', 205),
        (6, '2025-06-25', 'Conference on innovations in fintech and blockchain technology', 'https://example.com/fintech.jpg', 'Singapore', 'Fintech Future Summit', 206),
        (7, '2025-07-12', 'Gastronomy festival celebrating world cuisines and culture', 'https://example.com/foodfest.jpg', 'Tokyo', 'Global Food Festival', 207),
        (8, '2025-08-08', 'Concert featuring legendary rock bands from the 80s', 'https://example.com/rockconcert.jpg', 'Los Angeles', 'Rock Revival 2025', 208),
        (9, '2025-09-30', 'Startup competition with prizes for innovative projects', 'https://example.com/startup.jpg', 'Dubai', 'Startup Challenge 2025', 209),
        (10, '2025-10-20', 'Workshop on the latest trends in web development', 'https://example.com/webworkshop.jpg', 'Sydney', 'Web Dev Workshop 2025', 210)
    ]

    # Convert records into a list of strings (documents)
    documents = []
    for reservation in records:
        id, date, description, _, lieu, nom, utilisateur_id = reservation
        doc = f"Reservation ID: {id}, Date: {date}, Description: {description}, Location: {lieu}, Event Name: {nom}, Organizer ID: {utilisateur_id}"
        documents.append(doc)

    return documents


In [ ]:
def get_kv_cache(documents):
    print("Preloading reservations into the model's context...")
    combined_text = " ".join(documents)
    inputs = tokenizer(combined_text, return_tensors="pt", max_length=512, truncation=True)
    return inputs

In [ ]:
# user = input("User asa7bi")
# password = input("Password asa7bi")
# host = input("Host asa7bi")
# db_name = input("Database asa7bi")
reservations = fetch_reservations()
kv_cache = get_kv_cache(reservations)

Preloading reservations into the model's context...


**Using this function bellow for Infering without using a chatbot implementation**

In [ ]:
# def generate_reesponse(query, kv_cache):
#   input_text = f"Answer this question based on the context: {query} Context: {tokenizer.decode(kv_cache['input_ids'][0])}"
#   inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

#   outputs = model.generate(inputs["inputs_ids"], max_length=100)
#   response = tokenizer.decode(outputs[0], skip_spacial_token=True)

#   return response

**Inference**

In [ ]:

# Generate responses using the preloaded context
# def generate_response(query, kv_cache):
#     """
#     Generates a response using the preloaded KV cache (context).
#     """
#     print(f"Generating response for query: {query}")

#     # Combine the query with the preloaded context
#     input_text = f"Answer this question based on the context: {query} Context: {tokenizer.decode(kv_cache['input_ids'][0])}"
#     inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

#     # Generate the response
#     outputs = model.generate(inputs["input_ids"], max_length=100)
#     response = tokenizer.decode(outputs[0], skip_special_tokens=True)

#     return response

# # Step 2: Test the generate_response function
# def test_generate_response():
#     # Test 1: Query about events in San Francisco
#     query = "What events are happening in San Francisco?"
#     response = generate_response(query, kv_cache)
#     print("Test 1 - Events in San Francisco:")
#     print(response)

#     # Test 2: Query about the Global Music Fest
#     query = "What is the description of the Global Music Fest?"
#     response = generate_response(query, kv_cache)
#     print("Test 2 - Description of Global Music Fest:")
#     print(response)

#     # Test 3: Query about events organized by organizer ID 205
#     query = "Which event is organized by organizer ID 205?"
#     response = generate_response(query, kv_cache)
#     print("Test 3 - Event by Organizer ID 205:")
#     print(response)

#     # Test 4: Query about events in 2025
#     query = "What events are happening in 2025?"
#     response = generate_response(query, kv_cache)
#     print("Test 4 - Events in 2025:")
#     print(response)

# # Run the tests
# test_generate_response()

**Generate Response**

In [ ]:
def generate_response(query, kv_cache, chat_history=""):
    print(f"Generating response for query: {query}")

    # Combine the query, chat history, and preloaded context
    input_text = f"Chat History: {chat_history}\nQuestion: {query}\nContext: {tokenizer.decode(kv_cache['input_ids'][0])}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate the response
    outputs = model.generate(inputs["input_ids"], max_length=100)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response

**Chat bot inference**




In [ ]:
def reservation_chatbot():
    print("Welcome to the Reservation Chatbot! How can I assist you today?")
    chat_history = ""

    while True:
        # Get user input
        user_input = input("You: ")

        # Exit the chat if the user says "exit" or "quit"
        if user_input.lower() in ["exit", "quit"]:
            print("Chatbot: Goodbye! Have a great day.")
            break

        # Generate a response using the preloaded context and chat history
        response = generate_response(user_input, kv_cache, chat_history)
        print(f"Chatbot: {response}")

        # Update chat history with the latest interaction
        chat_history += f"\nYou: {user_input}\nChatbot: {response}"

# Run the chatbot
reservation_chatbot()

Welcome to the Reservation Chatbot! How can I assist you today?


KeyboardInterrupt: Interrupted by user

**Flask API**

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading

app = Flask(__name__)
# Set your ngrok authtoken
ngrok.set_auth_token("2rfb9JrW9pGK67aQQadvi48Xlx0_6DcHM728fCGEsQAu7ifMN")
print("Ngrok authtoken set successfully.")

@app.route('/chat', methods=['POST'])
def chat():
    # Get the user query from the request body
    data = request.json
    query = data.get('query')
    chat_history = data.get('chat_history', "")  # Get chat history if provided

    if not query:
        return jsonify({"error": "Query is required"}), 400

    # Generate a response using the preloaded context
    response = generate_response(query, kv_cache, chat_history)

    # Return the response as JSON
    return jsonify({"response": response})

# Function to run the Flask app
def run_flask():
    # Start ngrok tunnel for Flask
    flask_ngrok_tunnel = ngrok.connect(5000)
    flask_public_url = flask_ngrok_tunnel.public_url
    print(f" * Flask API is running at: {flask_public_url}")

    # Write the Flask ngrok URL to a file
    with open("flask_ngrok_url.txt", "w") as f:
        f.write(flask_public_url)

    # Run the Flask app (blocking call)
    app.run(host='0.0.0.0', port=5000, debug=True)  # Enable debug mode for detailed error logs

# Run Flask app in a separate thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

# Wait for Flask app to start
import time
time.sleep(5)


Ngrok authtoken set successfully.
 * Flask API is running at: https://9017-35-230-54-166.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.2:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-11 (run_flask):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-3f7440873bdf>", line 38, in run_flask
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 662, in run
    run_simple(t.cast(str, host), port, self, **options)
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/serving.py", line 1115, in run_simple
    run_with_reloader(
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/_reloader.py", line 452, in run_with_reloader
    signal.signal(signal.SI

**Chat Interface**

In [ ]:
%%writefile app.py
import streamlit as st
import requests

# Read the Flask ngrok URL from the file
with open("flask_ngrok_url.txt", "r") as f:
    flask_ngrok_url = f.read().strip()

# Define the Flask API endpoint using the ngrok URL
API_URL = f"{flask_ngrok_url}/chat"

# Streamlit app title
st.title("BS-Enventos Reservation Chatbot")

# Initialize session state to store chat history
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

# Function to send a query to the Flask API
def send_query(query):
    try:
        response = requests.post(API_URL, json={"query": query})
        if response.status_code == 200:
            return response.json().get("response", "No response from the chatbot.")
        else:
            return f"Error: {response.status_code} - {response.text}"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Display chat history
for message in st.session_state.chat_history:
    with st.chat_message(message["role"]):
        st.write(message["content"])

# Input field for user query
user_query = st.chat_input("Type your message here...")

# Handle user input
if user_query:
    st.session_state.chat_history.append({"role": "user", "content": user_query})
    with st.chat_message("user"):
        st.write(user_query)
    chatbot_response = send_query(user_query)
    st.session_state.chat_history.append({"role": "assistant", "content": chatbot_response})
    with st.chat_message("assistant"):
        st.write(chatbot_response)

Writing app.py


In [ ]:
# Run Streamlit app in the background
import subprocess
streamlit_process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8502"])

# Wait for Streamlit app to start
import time
time.sleep(5)

print("Streamlit app is running in the background.")

Streamlit app is running in the background.


In [ ]:
ngrok.kill()
print("All existing ngrok tunnels have been killed.")

All existing ngrok tunnels have been killed.


In [ ]:
from pyngrok import ngrok

# Expose Streamlit app with ngrok
streamlit_ngrok_tunnel = ngrok.connect(8502)
streamlit_public_url = streamlit_ngrok_tunnel.public_url
print(f" * Streamlit app is running at: {streamlit_public_url}")

 * Streamlit app is running at: https://7be0-35-230-54-166.ngrok-free.app
